In [48]:
from mortgage import Loan

In [49]:
loan = Loan(principal=1100000, interest=0.065, term=25)
loan.summarize

Original Balance:         $  1,100,000
Interest Rate:                   0.065 %
APY:                              6.70 %
APR:                              6.50 %
Term:                               25 years
Monthly Payment:          $    7427.28

Total principal payments: $1,100,000.00
Total interest payments:  $1,128,183.63
Total payments:           $2,228,183.63
Interest to principal:           102.6 %
Years to pay:                     25.0


In [50]:
class MyLoan:

    @staticmethod
    def recurring_payment(P, r, n):
        return P * (r * (1 + r)**n) / ((1 + r)**n - 1)

    def __init__(self, P, R, N, k = 12):
        self.P = P
        self.R = R
        self.N = N
        self.k = k

        self.r = R / k
        self.n = N * k
        self.p = self.recurring_payment(self.P, self.r, self.n)
        self.m = self.p * k / 12
    
    def print(self):
        print('--- Configuration ---')
        print('Principal P:', self.P)
        print('Interest R:', self.R * 100, 'ppa')
        print('Term length N:', self.N, 'yrs')
        print('Payments per year k:', self.k)
        print('')
        print('--- Analysis ---')
        print('Payment per month m:', f'{self.m:.2f}')
        print('Total amount to be paid:', f'{(self.n * self.p):.2f}')
        print('Total interest to be paid: ', f'{(self.n * self.p - self.P):.2f}', f'{((self.n * self.p - self.P) / self.P) * 100)):.2f}')
        print('')

    def simulate(self):
        print('--- Simulation ---')
        amount_owed = self.P
        time = 0
        while amount_owed > 0:
            time += 1
            interest_paid = amount_owed * self.r
            principal_paid = self.p - interest_paid
            amount_owed -= principal_paid
            print('time:', time, ', interest:', f'{(interest_paid):.2f}', ', principal: ', f'{(principal_paid):.2f}', ', owed:', f'{(amount_owed):.2f}')
        print('')

In [51]:
myLoan = MyLoan(P=1100000, R=0.065, N=25, k=12)
myLoan.print()
myLoan.simulate()

--- Configuration ---
Principal P: 1100000
Interest R: 6.5 ppa
Term length N: 25 yrs
Payments per year k: 12

--- Analysis ---
Payment per month m: 7427.28
Total amount to be paid: 2228183.63
Total interest to be paid:  1128183.63

--- Simulation ---
time: 1 , interest: 5958.33 , principal:  1468.95 , owed: 1098531.05
time: 2 , interest: 5950.38 , principal:  1476.90 , owed: 1097054.15
time: 3 , interest: 5942.38 , principal:  1484.90 , owed: 1095569.25
time: 4 , interest: 5934.33 , principal:  1492.95 , owed: 1094076.30
time: 5 , interest: 5926.25 , principal:  1501.03 , owed: 1092575.27
time: 6 , interest: 5918.12 , principal:  1509.16 , owed: 1091066.11
time: 7 , interest: 5909.94 , principal:  1517.34 , owed: 1089548.77
time: 8 , interest: 5901.72 , principal:  1525.56 , owed: 1088023.22
time: 9 , interest: 5893.46 , principal:  1533.82 , owed: 1086489.40
time: 10 , interest: 5885.15 , principal:  1542.13 , owed: 1084947.27
time: 11 , interest: 5876.80 , principal:  1550.48 , owed: